In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
import seaborn as sp
from torch.autograd import Variable
from torch import autograd
from datetime import datetime

In [19]:
# Generator class
from unicodedata import bidirectional


class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, kernel_size=3):
        super(Generator, self).__init__()

        self.encoder_lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

        self.encoder_conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size, padding=1)
        self.batchNorm1 = nn.BatchNorm1d(hidden_size)
        
        self.encoder_conv2 = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=kernel_size, padding=1)
        self.batchNorm2 = nn.BatchNorm1d(hidden_size)
        
        self.decoder_lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.decoder_linear = nn.Linear(hidden_size * 2, output_size)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # lstm_input = torch.transpose(x, 1, 2)
        lstm, (_,_) = self.encoder_lstm(x)
        lstm = self.dropout(lstm)
        conv_input = torch.transpose(x, 1, 2)
        encoded = self.encoder_conv1(conv_input)
        encoded = self.batchNorm1(encoded)
        
        # encoded = self.encoder_conv2(encoded)
        # encoded = self.batchNorm2(encoded)
        # decoded = self.activation(decoded)
        
        decoded = torch.transpose(encoded, 1, 2)

        decoded = (lstm + decoded) / 2
        final, _ = self.decoder_lstm(decoded)
        #final = self.dropout(decoded)
        final = self.decoder_linear(final)
        
        return self.activation(final)

In [18]:
# Discriminator class
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, kernel_size=3):
        super(Discriminator, self).__init__()

        self.encoder_lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, bidirectional=True)

        self.encoder_conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=kernel_size, padding=1)
        self.batchNorm1 = nn.BatchNorm1d(hidden_size)
        
        # self.encoder_conv2 = nn.Conv1d(in_channels=hidden_size * 2, out_channels=hidden_size, kernel_size=kernel_size, padding=1)
        # self.batchNorm2 = nn.BatchNorm1d(hidden_size)
        
        #self.decoder_lstm = nn.LSTM(input_size=hidden_size * 3, hidden_size=hidden_size, batch_first=True, bidirectional=True)
        self.decoder_linear1 = nn.Linear(input_size, hidden_size)
        self.decoder_linear = nn.Linear(2 * hidden_size * lag_size, output_size)
        self.activation = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        # lstm_input = torch.transpose(x, 1, 2)
        lstm, (_,_) = self.encoder_lstm(x)

        # conv_input = torch.transpose(x, 1, 2)
        # encoded = self.encoder_conv1(conv_input)
        # encoded = self.batchNorm1(encoded)
        
        # encoded = self.encoder_conv2(encoded)
        # encoded = self.batchNorm2(encoded)
        # decoded = self.activation(decoded)
        
        #decoded = torch.transpose(encoded, 1, 2)

        #lstm = torch.cat((lstm, decoded), -1)
        #final, _ = self.decoder_lstm(lstm)
        #lstm = self.dropout(lstm)
        lstm = self.flatten(lstm)
        # x = self.decoder_linear1(x)
        final = self.decoder_linear(lstm)
        
        return self.activation(final)